# Daily web scrape of stock prices
This code accesses site to retrieve stock price information daily ad add it to the h5 file. The steps are:
1. Get FTSE 100 and FTSE 250 lists from wikipedia, then compile into one list.
2. open pipe to h5 file holding current price data.
3. Loop though the tickers and for each one; 
    1. find the most recent price date.
    2. convert this into a time stamp to be used on Yahoo finance.
    3. go to Yahoo Finance and get all the prices between the last time stamp and the current timestamp.
4. Add these new prices to the h5 file.
5. Create a new (empty) h5 file and transfer all the data from the old file into the new one.
6. Delete the old h5 file and rename the new one.

The sources of data are:
- https://en.wikipedia.org/wiki/FTSE_100_Index -> list of FTSE 100 company stock tickers
- https://en.wikipedia.org/wiki/FTSE_250_Index -> list of FTSE 250 company stock tickers
- https://finance.yahoo.com/quote/{stock-ticker}/history?period1={start-time-mark}&period2={end-time-mark}&interval={interval}&filter=history&frequency={frequency} -> Example web address to retrieve information from Yahoo finance
    - Data on this page is scroll loaded so many time indexes must be used toretrieve the dcorrect data
    - Up to 145 records can be seen from initial page load note that this includes dividends so limit to 140 for safety

The inputs required for scrapping are:
 - {stock-ticker} -> this is the ticker taken from wiki with '.L' appended to it
 - {start-tme-mark} -> This is the time in seconds since 01/01/1970 at which you would like the data retrieval to start, data retrieved is inclusive of this time
 - {end-tme-mark} -> This is the time in seconds since 01/01/1970, data retrieved is inclusive of this time
 - {interval} & {frequency} -> This is the interval for which values are given, the two must match
     - 1d = 1 every 1 days
     - 1wk = 1 every week
     - 1mo = 1 eveery month

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup as bs
import requests as rq
import re
import datetime as dt
import os
import tables

In [2]:
#Set options for matplotlib
%matplotlib inline

In [3]:
#Clean column names
def clean_col_name(str_in):
    str_out = str_in.lower() #Change all to lower case
    str_out = re.sub(r'.*\.','',str_out) #Remove everything before a .
    str_out = re.sub(r' ','_',str_out) #Replace spaces with _
    str_out = re.sub(r'_?&_?','_and_',str_out) #Put _ around & and @ and change to 'and' and 'at'
    str_out = re.sub(r'_?@_?','_at_',str_out) #Put _ around & and @ and change to 'and' and 'at'
    str_out = re.sub(r'[^0-9a-z_]','',str_out) #Remove all non allowed characters
    str_out = re.sub(r'^_','',str_out) #Remove leading _
    return str_out

In [4]:
class process_time:
    def __init__(self,name:str = ''):
        self.st_time = dt.datetime.now()
        self.lap_li = []
        self.en_time = None
        self.name = name
    def calc_el_time(self,st_time,en_time):
        diff_time = en_time - st_time
        duration_in_s = diff_time.total_seconds()
        hours = int(divmod(duration_in_s, 3600)[0])
        duration_in_s += -(hours * 3600)
        minutes = int(divmod(duration_in_s, 60)[0])
        duration_in_s += -(minutes * 60)
        seconds = int(duration_in_s)
        return [hours,minutes,seconds]
    def lap(self):
        self.lap_li.append(dt.datetime.now())
    def end(self):
        self.en_time = dt.datetime.now()
        lap_time = self.calc_el_time(self.st_time,self.en_time)
        if self.name != '':
            print('TOTAL ELAPSED TIME OF {} -> {}:{}:{}'.format(self.name,lap_time[0],lap_time[1],lap_time[2]))
        else:
            print('TOTAL ELAPSED TIME -> {}:{}:{}'.format(lap_time[0],lap_time[1],lap_time[2]))
    def show_lap_times(self):
        tmp_count = 0
        for lap in self.lap_li:
            tmp_count += 1
            lap_time = self.calc_el_time(self.st_time,lap)
            print('LAP {} TIME -> {}:{}:{}'.format(tmp_count,lap_time[0],lap_time[1],lap_time[2]))
    def show_latest_lap_time(self):
        if len(self.lap_li) == 0:
            return
        elif len(self.lap_li) < 2:
            lap_time = self.calc_el_time(self.st_time,self.lap_li[-1])
        else:
            lap_time = self.calc_el_time(self.lap_li[-2],self.lap_li[-1])
        print('LAP {} TIME -> {}:{}:{}'.format(len(self.lap_li),lap_time[0],lap_time[1],lap_time[2]))

# Delete the old temporary files (if they exist)

In [5]:
#close any open h5 files
tables.file._open_files.close_all()

In [6]:
#Delete the old h5 files
src_fldr_pth = r'C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\'
try:
    os.remove(src_fldr_pth + r'all_hist_prices_d_TMP.h5')
    print('\nSUCCESSFULLY REMOVED {}'.format(src_fldr_pth + r'all_hist_prices_d_TMP.h5'))
except Exception as e:
    print('\nERROR - REMOVING:{}'.format(e))
try:
    os.remove(src_fldr_pth + r'all_hist_prices_w_TMP.h5')
    print('\nSUCCESSFULLY REMOVED {}'.format(src_fldr_pth + r'all_hist_prices_w_TMP.h5'))
except Exception as e:
    print('\nERROR - REMOVING:{}'.format(e))


SUCCESSFULLY REMOVED C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_d_TMP.h5

SUCCESSFULLY REMOVED C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_w_TMP.h5


# Scraping tickers
This section will scrap the ticker values for the FTSE 100 and FTSE 250 and store them in dataframes 'tick_ftse100' and 'tick_ftse250'.

Finally concatenate into 1 dataframe 'tick_ftse'.

In [7]:
#Fetch the data for ftse 100
web_add = 'https://en.wikipedia.org/wiki/FTSE_100_Index'
resp = rq.get(web_add)
parser = bs(resp.content,'html.parser')
#Get the table id=constituents
table = parser.find_all('table',id='constituents')[0]
#Get rows ignoring first row
rows = table.find_all('tr')[1:]
row_li = []
for r in rows:
    temp_row = []
    for r2 in r.find_all('td'):
        temp_row.append(re.sub('(?=[A-Z]*)\..*','',r2.text.upper().rstrip()))
    temp_row = temp_row[:2]
    row_li.append(temp_row)
#Create a dataframe
tick_ftse100 = pd.DataFrame(data=row_li,columns=['company','ticker'])
tick_ftse100['index'] = 'FTSE100'
tick_ftse100

,company,ticker,index
0,3I,III,FTSE100
1,ADMIRAL GROUP,ADM,FTSE100
2,ANGLO AMERICAN PLC,AAL,FTSE100
3,ANTOFAGASTA,ANTO,FTSE100
4,ASHTEAD GROUP,AHT,FTSE100
5,ASSOCIATED BRITISH FOODS,ABF,FTSE100
6,ASTRAZENECA,AZN,FTSE100
7,AUTO TRADER GROUP,AUTO,FTSE100
8,AVEVA,AVV,FTSE100
9,AVIVA,AV,FTSE100


In [8]:
#Fetch the data for ftse 250
web_add = 'https://en.wikipedia.org/wiki/FTSE_250_Index'
resp = rq.get(web_add)
parser = bs(resp.content,'html.parser')
#Get the table id=constituents
table = parser.find_all('table',id='constituents')[0]
#Get rows ignoring first row
rows = table.find_all('tr')[1:]
row_li = []
for r in rows:
    temp_row = []
    for r2 in r.find_all('td'):
        temp_row.append(re.sub('(?=[A-Z]*)\..*','',r2.text.upper().rstrip()))
    temp_row = temp_row[:2]
    row_li.append(temp_row)
#Create a dataframe
tick_ftse250 = pd.DataFrame(data=row_li,columns=['company','ticker'])
tick_ftse250['index'] = 'FTSE250'
tick_ftse250

,company,ticker,index
0,3I INFRASTRUCTURE,3IN,FTSE250
1,4IMPRINT,FOUR,FTSE250
2,ABERFORTH SMALLER COMPANIES TRUST,ASL,FTSE250
3,AGGREKO,AGK,FTSE250
4,AIRTEL AFRICA,AAF,FTSE250
5,ALLIANCE TRUST,ATST,FTSE250
6,APAX GLOBAL ALPHA,APAX,FTSE250
7,ASCENTIAL,ASCL,FTSE250
8,ASHMORE GROUP,ASHM,FTSE250
9,ASSURA,AGR,FTSE250


In [9]:
#Combine into 1 dataframe
tick_ftse = pd.concat([tick_ftse100,tick_ftse250])
tick_ftse

,company,ticker,index
0,3I,III,FTSE100
1,ADMIRAL GROUP,ADM,FTSE100
2,ANGLO AMERICAN PLC,AAL,FTSE100
3,ANTOFAGASTA,ANTO,FTSE100
4,ASHTEAD GROUP,AHT,FTSE100
5,ASSOCIATED BRITISH FOODS,ABF,FTSE100
6,ASTRAZENECA,AZN,FTSE100
7,AUTO TRADER GROUP,AUTO,FTSE100
8,AVEVA,AVV,FTSE100
9,AVIVA,AV,FTSE100


# Get the latest price dates
Get the latest price file.

Loop through the tickers in tick_ftse and for each one get the latest date of scrape.

Convert this date into a timestamp and add to the dataframe.

In [10]:
#open the price file
hist_prices_df = pd.read_hdf(src_fldr_pth + r'all_hist_prices_d.h5')
hist_prices_df.head()

,ticker,date,open,close,high,low,change,volume,ema12,ema26,macd_line,signal,macd
0,III,2008-01-02,995.0,1005.0,1023.0,995.0,10.0,785944.0,NaN,NaN,NaN,NaN,NaN
1,III,2008-01-03,1005.0,988.0,1020.0,981.0,-17.0,1805560.0,NaN,NaN,NaN,NaN,NaN
2,III,2008-01-04,985.5,965.0,1000.0,964.0,-20.5,1920061.0,NaN,NaN,NaN,NaN,NaN
3,III,2008-01-07,967.5,971.5,974.0,953.0,4.0,3167216.0,NaN,NaN,NaN,NaN,NaN
4,III,2008-01-08,975.0,978.5,989.0,971.5,3.5,2297118.0,NaN,NaN,NaN,NaN,NaN


In [11]:
#Convert date
def conv_date(_str_in):
    if type(_str_in) == str:
        return dt.datetime.strptime(_str_in,'%Y-%m-%d')
    else:
        return _str_in
hist_prices_df.date = [conv_date(x) for x in hist_prices_df.date]
hist_prices_df.dtypes

ticker               object
date         datetime64[ns]
open                float64
close               float64
high                float64
low                 float64
change              float64
volume              float64
ema12               float64
ema26               float64
macd_line           float64
signal              float64
macd                float64
dtype: object

# Scraping a stock history
Working backwards through time from now until the last price scrape date collect all the daily data for a stock. Daily is used as it can later be summarised into weekly or monthly if required.

This will then be put into a dataframe containing:
- ticker
- Company name
- date
- open
- high
- low
- close
- Adjusted close (for divs and splits)
- volume

In [12]:
#Create a list of time intervals to be used with 140 days in each item
def create_sec_ref_li(_st_date:int,_en_date:int):
    #Establish the day ref of the dates compared to 01/01/1970
    _ep_date = pd.to_datetime(dt.datetime(1970,1,1),errors='coerce')
    _en_date = pd.to_datetime(_en_date,errors='coerce')
    _st_date = pd.to_datetime(_st_date,errors='coerce')
    print('_st_date: ' + str(_st_date))
    print('_en_date: ' + str(_en_date))
    _st_days = (_st_date - _ep_date).days
    _en_days = (_en_date - _ep_date).days
    #Loop adding to a list until reaching 0
    _sec_ref_li = []
    _days = 140
    while _en_days > _st_days:
        if _en_days - _days > _st_days:
            _sec_ref_li.append([(_en_days - _days)*86400,_en_days*86400])
        else:
            _sec_ref_li.append([_st_days*86400,_en_days*86400])        
        _en_days += -_days
    return _sec_ref_li

In [13]:
def calc_ema(s_in,periods):
    #Calc mod val
    mod = 2/(periods+1)
    #Make a df
    tmp_df = pd.DataFrame(s_in)
    #Calc sma
    tmp_df['sma'] = pd.Series([0] * len(s_in))
    for i in range(0,periods):
        tmp_df['sma'] += tmp_df[s_in.name].shift(i) / periods
    #Calc ema
    tmp_df['ema'] = tmp_df['sma'].copy()
    for row in tmp_df.iterrows():
        i = row[0]
        if i > 0 and not np.isnan(tmp_df['sma'][i-1]):
            tmp_df['ema'][i] = mod*(tmp_df[s_in.name][i] - tmp_df['ema'][i-1]) + tmp_df['ema'][i-1]
    return tmp_df['ema'].copy()

In [14]:
def calc_macd(ema_lng_s,ema_sht_s,sig_period):
    #Make a df
    tmp_df = pd.DataFrame([])
    tmp_df['ema_lng'] = ema_lng_s
    tmp_df['ema_sht'] = ema_sht_s
    #Calc the signal line
    tmp_df['macd_line'] = tmp_df['ema_sht'] - tmp_df['ema_lng']
    tmp_df['signal_line'] = calc_ema(tmp_df['macd_line'],sig_period)
    tmp_df['macd_hist'] = tmp_df['macd_line'] - tmp_df['signal_line']
    return (tmp_df['macd_line'].copy(),tmp_df['signal_line'].copy(),tmp_df['macd_hist'].copy())

In [15]:
#Create a dictionary of max character lengths of fields for use later in h5 file appending
def get_col_lens(_df_in):
    _col_lens = {}
    for c in _df_in:
        _tmp_s = pd.Series([len(str(x)) for x in _df_in[c]])
        _col_lens[c] = _tmp_s.max()
    return _col_lens
col_lens = get_col_lens(hist_prices_df)
# col_lens = {'ticker': 4,
#  'date': 19,
#  'open': 8,
#  'close': 8,
#  'high': 8,
#  'low': 8,
#  'change': 22,
#  'volume': 12,
#  'ema12': 18,
#  'ema26': 18,
#  'macd_line': 23,
#  'signal_line': 23,
#  'macd': 23}
col_lens

{'ticker': 4,
 'date': 19,
 'open': 8,
 'close': 8,
 'high': 8,
 'low': 8,
 'change': 22,
 'volume': 12,
 'ema12': 18,
 'ema26': 18,
 'macd_line': 23,
 'signal_line': 23,
 'macd': 23}

In [16]:
#Get the price history for a specific ticker
def get_price_hist_d(_tick:str,_sec_ref_li:list):
    try:
        _tick = re.sub('[^A-Z0-9\-]','',_tick)
        print('Getting DAILY prices for:{}'.format(_tick))
        _tick_df = pd.DataFrame([])
        _cols = []

        for _secs in _sec_ref_li:
            try:
                _web_add = 'https://finance.yahoo.com/quote/{0}/history?period1={1}&period2={2}&interval={3}&filter=history&frequency={3}'.format(_tick+'.L',_secs[0],_secs[1],'1d')
#                 print('_web_add: {}'.format(_web_add))
                
                _resp = rq.get(_web_add)
                if int(_resp.status_code) != 200:
                    print('status code: {}'.format(_resp.status_code))
                    continue
                _parser = bs(_resp.content,'html.parser')
                #Get the table
                _table = _parser.find_all('table',attrs={'data-test':'historical-prices'})[0]
                #Grab the data rows
                _rows = _table.find_all('tbody')[0].find_all('tr')
                #Put the rows into the dataframe
                for _r in _rows:
                    if len(_tick_df) == 0:
                        _cols = [clean_col_name(x.text) for x in _table.find_all('th')]
                        _tick_df = pd.DataFrame([],columns=_cols)
                    if len(_r.find_all('td')) == len(_cols):
                        _tick_df = _tick_df.append(pd.Series([x.text for x in _r.find_all('td')],index=_cols),ignore_index=True)
                    else:
                        continue
            except Exception as e:
                print('ERROR - CONTINUE:{}'.format(e))
                continue
        #Check for rows - if none then return
        if len(_tick_df) == 0:
            return _tick_df
        #Reformat
        def float_format(_str_in):
            if type(_str_in) == str:
                _str_in = _str_in.strip()
                _str_in = re.sub('[^0-9.]','',_str_in)
                if _str_in == '':
                    _str_in = 0
                return _str_in
            else:
                return _str_in
        _tick_df.loc[:,'open'] = _tick_df.loc[:,'open'].apply(float_format).astype(float)
        _tick_df.loc[:,'high'] = _tick_df.loc[:,'high'].apply(float_format).astype(float)
        _tick_df.loc[:,'low'] = _tick_df.loc[:,'low'].apply(float_format).astype(float)
        _tick_df.loc[:,'close'] = _tick_df.loc[:,'close'].apply(float_format).astype(float)
        _tick_df.loc[:,'adj_close'] = _tick_df.loc[:,'adj_close'].apply(float_format).astype(float)
        _tick_df.loc[:,'volume'] = _tick_df.loc[:,'volume'].apply(float_format).astype(float)
        _tick_df.loc[:,'change'] = _tick_df.loc[:,'close'] - _tick_df.loc[:,'open']
        def conv_date(_str_in):
            if type(_str_in) == str:
                return dt.datetime.strptime(_str_in,'%b %d, %Y')
            else:
                return _str_in
        _tick_df.loc[:,'date'] = _tick_df.loc[:,'date'].apply(conv_date)
        #Add the ticker series
        _tick_df.loc[:,'ticker'] = _tick    
        _tick_df = _tick_df.loc[:,['ticker','date','open','close','high','low','change','volume']]
        #CLEANING - Remove any rows with zero volume
        _tick_df = _tick_df[_tick_df['volume'] > 0]
        #CLEANING - Copy row above where the change has been more than 90%
        _tick_df['cl_change'] = (_tick_df['close'] - _tick_df['close'].shift(1))/_tick_df['close'].shift(1)
        _check_s = _tick_df['cl_change'] < -0.9
        _tick_df.loc[_check_s,'open'] = _tick_df['open'].shift(-1).copy().loc[_check_s]
        _tick_df.loc[_check_s,'close'] = _tick_df['close'].shift(-1).copy().loc[_check_s]
        _tick_df.loc[_check_s,'high'] = _tick_df['high'].shift(-1).copy().loc[_check_s]
        _tick_df.loc[_check_s,'low'] = _tick_df['low'].shift(-1).copy().loc[_check_s]
        _tick_df.drop(columns='cl_change',inplace=True)
        return _tick_df
    except Exception as e:
        print('ERROR:{}'.format(e))
        return False

#Calc the ema and macds for the data
def calc_ema_macd(_tick_df):
    try:
        #Add in the ema and macd
        _tick_df = _tick_df.sort_values(by='date')
        _tick_df = _tick_df.reset_index(drop=True)
        _tick_df['ema12'] = calc_ema(_tick_df['close'],12)
        _tick_df['ema26'] = calc_ema(_tick_df['close'],26)
        _tick_df['macd_line'],_tick_df['signal_line'],_tick_df['macd'] = calc_macd(_tick_df['ema26'],_tick_df['ema12'],9)
        #Sort clean and export
        _tick_df = _tick_df.sort_values(by='date')
        _tick_df = _tick_df.reset_index(drop=True)
        return _tick_df
    except Exception as e:
        print('ERROR:{}'.format(e))
        return False

In [17]:
#Create a weekly table
def get_price_hist_w(_df_d):
    print('Converting daily prices to weekly prices')
    try:
        #Create a copy of the data
        _df_d = _df_d.copy()
        #Establish a week number for each date
        _df_d['isocalendar'] = [x.isocalendar()[:2] for x in _df_d['date']]
        #Get highs and lows
        _high_df = _df_d.loc[_df_d['high'] > 0,['high','isocalendar']].groupby('isocalendar').max().reset_index()
        _low_df = _df_d.loc[_df_d['low'] > 0,['low','isocalendar']].groupby('isocalendar').min().reset_index()
        #Get total volume for the week
        _vol_df = _df_d.loc[_df_d['volume'] > 0,['volume','isocalendar']].groupby('isocalendar').sum().reset_index()
        #Get open price
        _min_wk_day = _df_d.loc[_df_d['open'] > 0,['date','isocalendar']].groupby('isocalendar').min().reset_index()
        _open_df = pd.merge(_df_d[['date','open']],_min_wk_day,left_on='date',right_on='date')
        #Get close price
        _max_wk_day = _df_d.loc[_df_d['close'] > 0,['date','isocalendar']].groupby('isocalendar').max().reset_index()
        _close_df = pd.merge(_df_d[['date','close']],_max_wk_day,left_on='date',right_on='date').reset_index()
        #Form the final df
        _wk_df = pd.merge(_df_d[['ticker','isocalendar']],_min_wk_day,left_on='isocalendar',right_on='isocalendar') #date
        _wk_df = pd.merge(_wk_df,_high_df,left_on='isocalendar',right_on='isocalendar') #high
        _wk_df = pd.merge(_wk_df,_low_df,left_on='isocalendar',right_on='isocalendar') #low
        _wk_df = pd.merge(_wk_df,_vol_df,left_on='isocalendar',right_on='isocalendar') #volume
        _wk_df = pd.merge(_wk_df,_open_df[['isocalendar','open']],left_on='isocalendar',right_on='isocalendar') #open
        _wk_df = pd.merge(_wk_df,_close_df[['isocalendar','close']],left_on='isocalendar',right_on='isocalendar') #close
        _wk_df['change'] = _wk_df['close'] - _wk_df['open']
        _wk_df = _wk_df.drop_duplicates().reset_index(drop=True)
        #Get the monday of each week
        _wk_df['weekday'] = [dt.date.weekday(x) for x in _wk_df['date']]
        _wk_df['date'] = _wk_df['date'] - pd.Series([dt.timedelta(days=x) for x in _wk_df['weekday']])
        _wk_df.drop(columns=['isocalendar','weekday'],inplace=True)
        return _wk_df
    except Exception as e:
        print('ERROR:{}'.format(e))
        return False

In [18]:
#Scrape daily price data
hf_store_name_d = src_fldr_pth + r'all_hist_prices_d_TMP.h5'
hf_d = pd.HDFStore(hf_store_name_d)
group_name_d = r'daily_data'
hf_store_name_w = src_fldr_pth + r'all_hist_prices_w_TMP.h5'
hf_w = pd.HDFStore(hf_store_name_w)
group_name_w = r'weekly_data'
out_cols = ['ticker','date','open','close','high','low','change','volume']
count = 0
errors = []
run_time = process_time()
for tick in tick_ftse['ticker']:
    try:
        run_time.show_latest_lap_time()
        run_time.lap()
        count += 1
        print('\n{} RUNNING FOR: {}'.format(count,tick))
        tick_df = hist_prices_df[hist_prices_df.ticker == tick][out_cols]
        print('DAILY SHAPE BEFORE: {}'.format(tick_df.shape))
        
        #DAILY PRICES
        #Get last scrapped date
        st_date = tick_df.date.max()
        if pd.isnull(st_date):
            #Treat as a new share
            st_date = dt.datetime(1970,1,1)
        
        #Get new price data
        new_tick_df = get_price_hist_d(tick,create_sec_ref_li(st_date,dt.datetime.today()+dt.timedelta(days=1))) #1 day ahead so today is included
        
        #Join onto existing data
        tick_df = tick_df.append(new_tick_df)
        
        #Drop duplicates
        tick_df = tick_df.drop_duplicates()

        #Calc emas and macd
        tick_df = calc_ema_macd(tick_df)
        print('DAILY FINAL SHAPE: {}'.format(tick_df.shape))
        
        #Clarify col_lens with cur cols in data
        col_lens_tmp = {}
        for col in tick_df:
            if col in col_lens:
                col_lens_tmp[col] = col_lens[col]
                
        #Add to daily h5 file
        tick_df.to_hdf(hf_store_name_d,key=group_name_d,append=True,min_itemsize=col_lens_tmp)
        
        #WEEKLY PRICES
        #Convert to weekly prices
        df_w = get_price_hist_w(tick_df)
        
        #Drop duplicates
        df_w = df_w.drop_duplicates()

        #Calc emas and macd
        df_w = calc_ema_macd(df_w)
        print('WEEKLY FINAL SHAPE: {}'.format(df_w.shape))

        #Add to weekly h5 file
        df_w.to_hdf(hf_store_name_w,key=group_name_w,append=True,min_itemsize=col_lens)
    except Exception as e:
        print('ERROR:{}'.format(e))
        errors.append(e)
hf_d.close()
hf_w.close()
print('\n\n')
run_time.end()
print('\nERROR COUNT: {}'.format(len(errors)))
if len(errors) > 0:
    print('    ERRORS -> {}'.format(errors))


1 RUNNING FOR: III
DAILY SHAPE BEFORE: (2951, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:07:32.293414
Getting DAILY prices for:III
DAILY FINAL SHAPE: (2952, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (614, 13)
LAP 1 TIME -> 0:0:0

2 RUNNING FOR: ADM
DAILY SHAPE BEFORE: (1751, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:07:37.431312
Getting DAILY prices for:ADM
DAILY FINAL SHAPE: (1753, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (372, 13)
LAP 2 TIME -> 0:0:5

3 RUNNING FOR: AAL
DAILY SHAPE BEFORE: (4322, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:07:40.720029
Getting DAILY prices for:AAL
DAILY FINAL SHAPE: (4323, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1015, 13)
LAP 3 TIME -> 0:0:3

4 RUNNING FOR: ANTO
DAILY SHAPE BEFORE: (5082, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:07:47.317979
Getting DAILY prices for:ANTO
DAILY FINAL SHAPE: (5083, 13)
Converting daily pri

DAILY FINAL SHAPE: (3283, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (677, 13)
LAP 31 TIME -> 0:0:4

32 RUNNING FOR: FERG
DAILY SHAPE BEFORE: (5082, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:11:56.323855
Getting DAILY prices for:FERG
DAILY FINAL SHAPE: (5084, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1053, 13)
LAP 32 TIME -> 0:0:5

33 RUNNING FOR: FLTR
DAILY SHAPE BEFORE: (88, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:12:04.832947
Getting DAILY prices for:FLTR
DAILY FINAL SHAPE: (89, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (17, 13)
LAP 33 TIME -> 0:0:8

34 RUNNING FOR: FRES
DAILY SHAPE BEFORE: (2879, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:12:06.450545
Getting DAILY prices for:FRES
DAILY FINAL SHAPE: (2881, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (596, 13)
LAP 34 TIME -> 0:0:1

35 RUNNING FOR: GSK
DAILY SHAPE BEFORE: (5034, 8)
_st_date: 2019-09-

LAP 61 TIME -> 0:0:8

62 RUNNING FOR: NMC
DAILY SHAPE BEFORE: (1895, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:15:10.622147
Getting DAILY prices for:NMC
DAILY FINAL SHAPE: (1896, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (391, 13)
LAP 62 TIME -> 0:0:7

63 RUNNING FOR: OCDO
DAILY SHAPE BEFORE: (2325, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:15:15.336468
Getting DAILY prices for:OCDO
DAILY FINAL SHAPE: (2326, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (480, 13)
LAP 63 TIME -> 0:0:4

64 RUNNING FOR: PSON
DAILY SHAPE BEFORE: (5077, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:15:20.327650
Getting DAILY prices for:PSON
DAILY FINAL SHAPE: (5078, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1053, 13)
LAP 64 TIME -> 0:0:4

65 RUNNING FOR: PSN
DAILY SHAPE BEFORE: (5064, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:15:29.906880
Getting DAILY prices for:PSN
DAILY FINAL SHAPE: (

DAILY FINAL SHAPE: (5019, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1058, 13)
LAP 92 TIME -> 0:0:6

93 RUNNING FOR: TW
DAILY SHAPE BEFORE: (5077, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:18:59.752446
Getting DAILY prices for:TW
DAILY FINAL SHAPE: (5078, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1053, 13)
LAP 93 TIME -> 0:0:8

94 RUNNING FOR: TSCO
DAILY SHAPE BEFORE: (5058, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:19:10.363744
Getting DAILY prices for:TSCO
DAILY FINAL SHAPE: (5059, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1053, 13)
LAP 94 TIME -> 0:0:10

95 RUNNING FOR: TUI
DAILY SHAPE BEFORE: (1211, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:19:19.994720
Getting DAILY prices for:TUI
DAILY FINAL SHAPE: (1212, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (250, 13)
LAP 95 TIME -> 0:0:9

96 RUNNING FOR: ULVR
DAILY SHAPE BEFORE: (5081, 8)
_st_date: 2019-

WEEKLY FINAL SHAPE: (246, 13)
LAP 122 TIME -> 0:0:4

123 RUNNING FOR: BGEO
DAILY SHAPE BEFORE: (2954, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:21:59.499794
Getting DAILY prices for:BGEO
DAILY FINAL SHAPE: (2956, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (618, 13)
LAP 123 TIME -> 0:0:3

124 RUNNING FOR: BEZ
DAILY SHAPE BEFORE: (4205, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:22:05.367743
Getting DAILY prices for:BEZ
DAILY FINAL SHAPE: (4206, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (882, 13)
LAP 124 TIME -> 0:0:5

125 RUNNING FOR: AJB
DAILY SHAPE BEFORE: (207, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:22:12.513130
Getting DAILY prices for:AJB
DAILY FINAL SHAPE: (208, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (43, 13)
LAP 125 TIME -> 0:0:7

126 RUNNING FOR: BWY
DAILY SHAPE BEFORE: (4435, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:22:15.385179
Getting DAILY pri

...

Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (675, 13)
LAP 213 TIME -> 0:0:6

214 RUNNING FOR: HSV
DAILY SHAPE BEFORE: (4969, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:31:44.784359
Getting DAILY prices for:HSV
DAILY FINAL SHAPE: (4970, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1041, 13)
LAP 214 TIME -> 0:0:5

215 RUNNING FOR: HWDN
DAILY SHAPE BEFORE: (5075, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:31:52.726197
Getting DAILY prices for:HWDN
DAILY FINAL SHAPE: (5076, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1053, 13)
LAP 215 TIME -> 0:0:7

216 RUNNING FOR: HTG
DAILY SHAPE BEFORE: (4609, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:31:59.817115
Getting DAILY prices for:HTG
DAILY FINAL SHAPE: (4610, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1021, 13)
LAP 216 TIME -> 0:0:7

217 RUNNING FOR: IBST
DAILY SHAPE BEFORE: (998, 8)
_st_date: 2019-09-27 00:00:00
_en_dat

WEEKLY FINAL SHAPE: (796, 13)
LAP 243 TIME -> 0:0:8

244 RUNNING FOR: MCS
DAILY SHAPE BEFORE: (987, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:34:41.453763
Getting DAILY prices for:MCS
DAILY FINAL SHAPE: (988, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (204, 13)
LAP 244 TIME -> 0:0:6

245 RUNNING FOR: MDC
DAILY SHAPE BEFORE: (918, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:34:44.882376
Getting DAILY prices for:MDC
DAILY FINAL SHAPE: (919, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (189, 13)
LAP 245 TIME -> 0:0:3

246 RUNNING FOR: MRC
DAILY SHAPE BEFORE: (2731, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:34:48.346706
Getting DAILY prices for:MRC
DAILY FINAL SHAPE: (2732, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (591, 13)
LAP 246 TIME -> 0:0:3

247 RUNNING FOR: MERL
DAILY SHAPE BEFORE: (1492, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:34:53.744129
Getting DAILY price

DAILY FINAL SHAPE: (1386, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (286, 13)
LAP 274 TIME -> 0:0:5

275 RUNNING FOR: PPH
DAILY SHAPE BEFORE: (2261, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:37:28.763126
Getting DAILY prices for:PPH
DAILY FINAL SHAPE: (2263, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (591, 13)
LAP 275 TIME -> 0:0:3

276 RUNNING FOR: PMO
DAILY SHAPE BEFORE: (5080, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:37:33.835520
Getting DAILY prices for:PMO
DAILY FINAL SHAPE: (5081, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1050, 13)
LAP 276 TIME -> 0:0:5

277 RUNNING FOR: PHP
DAILY SHAPE BEFORE: (4981, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:37:41.514569
Getting DAILY prices for:PHP
DAILY FINAL SHAPE: (4982, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1050, 13)
LAP 277 TIME -> 0:0:7

278 RUNNING FOR: PFG
DAILY SHAPE BEFORE: (5082, 8)
_st_date: 

WEEKLY FINAL SHAPE: (1037, 13)
LAP 304 TIME -> 0:0:7

305 RUNNING FOR: SHI
DAILY SHAPE BEFORE: (4974, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:40:36.272158
Getting DAILY prices for:SHI
DAILY FINAL SHAPE: (4975, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1047, 13)
LAP 305 TIME -> 0:0:7

306 RUNNING FOR: SXX
DAILY SHAPE BEFORE: (2974, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:40:44.371925
Getting DAILY prices for:SXX
DAILY FINAL SHAPE: (2975, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (677, 13)
LAP 306 TIME -> 0:0:8

307 RUNNING FOR: SRE
DAILY SHAPE BEFORE: (2740, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:40:49.767451
Getting DAILY prices for:SRE
DAILY FINAL SHAPE: (2741, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (642, 13)
LAP 307 TIME -> 0:0:5

308 RUNNING FOR: SSON
DAILY SHAPE BEFORE: (176, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:40:55.227700
Getting DAILY 

DAILY FINAL SHAPE: (5045, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1053, 13)
LAP 335 TIME -> 0:0:8

336 RUNNING FOR: VSVS
DAILY SHAPE BEFORE: (5087, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:43:52.707882
Getting DAILY prices for:VSVS
DAILY FINAL SHAPE: (5088, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1052, 13)
LAP 336 TIME -> 0:0:8

337 RUNNING FOR: VCT
DAILY SHAPE BEFORE: (4456, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:44:01.380393
Getting DAILY prices for:VCT
DAILY FINAL SHAPE: (4457, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (1018, 13)
LAP 337 TIME -> 0:0:8

338 RUNNING FOR: VEIL
DAILY SHAPE BEFORE: (759, 8)
_st_date: 2019-09-27 00:00:00
_en_date: 2019-09-30 08:44:09.379837
Getting DAILY prices for:VEIL
DAILY FINAL SHAPE: (760, 13)
Converting daily prices to weekly prices
WEEKLY FINAL SHAPE: (164, 13)
LAP 338 TIME -> 0:0:8

339 RUNNING FOR: VOF
DAILY SHAPE BEFORE: (2940, 8)
_st_dat

In [19]:
#close any open h5 files
tables.file._open_files.close_all()

In [20]:
#Delete the old h5 file and rename the TMP
try:
    os.remove(src_fldr_pth + r'all_hist_prices_d.h5')
    print('\nSUCCESSFULLY REMOVED {}'.format(src_fldr_pth + r'all_hist_prices_d.h5'))
except Exception as e:
    print('\nERROR - REMOVING:{}'.format(e))
try:
    os.rename(src_fldr_pth + r'all_hist_prices_d_TMP.h5',src_fldr_pth + r'all_hist_prices_d.h5')
    print('\nSUCCESSFULLY RENAMED {} TO {}'.format(src_fldr_pth + r'all_hist_prices_d_TMP.h5',src_fldr_pth + r'all_hist_prices_d.h5'))
except Exception as e:
    print('\nERROR - RENAMING:{}'.format(e))


SUCCESSFULLY REMOVED C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_d.h5

SUCCESSFULLY RENAMED C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_d_TMP.h5 TO C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_d.h5


In [21]:
#Delete the old h5 file and rename the TMP
try:
    os.remove(src_fldr_pth + r'all_hist_prices_w.h5')
    print('\nSUCCESSFULLY REMOVED {}'.format(src_fldr_pth + r'all_hist_prices_w.h5'))
except Exception as e:
    print('\nERROR - REMOVING:{}'.format(e))
try:
    os.rename(src_fldr_pth + r'all_hist_prices_w_TMP.h5',src_fldr_pth + r'all_hist_prices_w.h5')
    print('\nSUCCESSFULLY RENAMED {} TO {}'.format(src_fldr_pth + r'all_hist_prices_w_TMP.h5',src_fldr_pth + r'all_hist_prices_w.h5'))
except Exception as e:
    print('\nERROR - RENAMING:{}'.format(e))


SUCCESSFULLY REMOVED C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_w.h5

SUCCESSFULLY RENAMED C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_w_TMP.h5 TO C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\all_hist_prices_w.h5


# Exporting the FTSE ticker

In [22]:
#Export the ftse list
path = "C:\\Users\\Robert\\Documents\\python_scripts\\stock_trading_ml_modelling\\historical_prices\\"
tick_ftse.to_csv(path_or_buf=path + "tick_ftse.csv")

In [23]:
#End the program
import sys
sys.exit()

SystemExit: 

C:\Users\Robert\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
